In [11]:
from sklearn.linear_model import LogisticRegressionCV
import pandas as pd
import numpy as np

In [13]:
for data in ["antenatal"]:    
    for outcome in ['_hie']:
        
        print("Running LR on {} for {}".format(data, outcome))
        
        # read in data
        train = pd.read_csv("data/{}{}_train.csv".format(data, outcome), index_col=0).astype('float32')
        test = pd.read_csv("data/{}{}_test.csv".format(data, outcome), index_col=0).astype('float32')
        train_y = train.pop(outcome)
        test_y = test.pop(outcome)
        
        # evaluate model
        clf = LogisticRegressionCV(cv=5, random_state=0, penalty="l1", solver="saga", njobs=-1, max_iter=100000)
        clf.fit(train, train_y)

Running LR on antenatal for _hie


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


KeyboardInterrupt: 

In [ ]:
        # calculate probabilities for test data
        y_test_pred = clf.predict_proba(test)[:, 1]
    
        # calculate roc auc metric
        fpr, tpr, thresholds = roc_curve(test_y, y_test_pred)
        roc_auc = auc(fpr, tpr)
    
        # calculate brier loss for probability accuracy
        brier_loss = brier_score_loss(test_y, y_test_pred)
        
        # write out probs for delong test to obtain CIs (R)
        pd.DataFrame({"Prob" : y_test_pred, "{}{}".format(data, outcome): test_y}, index=test.index).to_csv("data/{}{}.LR.csv".format(data, outcome))
    
        print("ROC AUC: {}\nBrier loss: {}".format(np.round(roc_auc, 3), np.round(brier_loss, 3)))        